In [ ]:
import json

text = ""
filename = "./conll2003/train.txt"
with open(filename, 'r') as file:
    while True:
        temp = file.readline().split(" ")[0]
        if not temp:
            break
        text += " " + temp

print(text)